In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('../')
import argparse
import logging
import time
from os.path import join
import warnings
warnings.filterwarnings("ignore")

import uproot
from coffea import processor, util
from coffea.lumi_tools import LumiMask
from coffea.nanoevents import NanoAODSchema, NanoEventsFactory

from azh_analysis.processors.analysis_processor import AnalysisProcessor
from azh_analysis.utils.btag import get_btag_SFs, get_btag_tables
from azh_analysis.utils.corrections import (
    dyjets_stitch_weights,
    get_electron_ES_weights,
    get_electron_ID_weights,
    get_electron_trigger_SFs,
    get_fake_rates,
    get_met_phi_weights,
    get_muon_ES_weights,
    get_muon_ID_weights,
    get_muon_trigger_SFs,
    get_pileup_weights,
    get_tau_ID_weights,
)
from azh_analysis.utils.sample import get_fileset, get_nevts_dict, get_sample_info

# setup logging
log_format = "%(asctime)s %(levelname)s %(message)s"
logging.basicConfig(level=logging.INFO, format=log_format)
logging.info("Initializing")

# relevant parameters
year, source = '2018', 'signal_UL'

# load up golden jsons
golden_json_dir = "../samples/data_certification"
golden_jsons = {
    "2018": join(golden_json_dir, "data_cert_2018.json"),
    "2017": join(golden_json_dir, "data_cert_2017.json"),
    "2016postVFP": join(golden_json_dir, "data_cert_2016.json"),
    "2016preVFP": join(golden_json_dir, "data_cert_2016.json"),
}
lumi_masks = {year: LumiMask(golden_json) for year, golden_json in golden_jsons.items()}

# load up fake rates
fr_base = f"../corrections/fake_rates/UL_{year}"
fake_rates = get_fake_rates(fr_base, year)
logging.info(f"Using fake rates\n{fr_base}")

# get the met phi weights                                                                                                    
met_phi_base = f"../corrections/met/{year}"
met_phi_file = join(met_phi_base, "met.json.gz")
met_phi_SFs = get_met_phi_weights(met_phi_file)
logging.info(f"Using met_phi_weights:\n{met_phi_file}")

# load up electron / muon / tau IDs
eID_base = f"../corrections/electron_ID/UL_{year}"
eID_file = join(
    eID_base, f"Electron_RunUL{year}_IdIso_AZh_IsoLt0p15_IdFall17MVA90noIsov2.root"
)

eIDs = get_electron_ID_weights(eID_file)
logging.info(f"Using eID_SFs:\n{eID_file}")

eES_SFs = get_electron_ES_weights("../corrections/electron_ES/", year)
logging.info(f"Using eES_SFs from corrections/ele_ES/UL_{year}")

mES_SFs = get_muon_ES_weights("../corrections/muon_ES/", year)
logging.info(f"Using mES_SFs from corrections/muon_ES/UL_{year}")

mID_base = f"../corrections/muon_ID/UL_{year}"
mID_file = join(mID_base, f"Muon_RunUL{year}_IdIso_AZh_IsoLt0p15_IdLoose.root")
mIDs = get_muon_ID_weights(mID_file)
logging.info(f"Using mID_SFs:\n{mID_file}")

tID_base = f"../corrections/tau_ID/UL_{year}"
tID_file = join(tID_base, "tau.corr.json")
tIDs = get_tau_ID_weights(tID_file)
logging.info(f"Using tID_SFs:\n{tID_file}")

# load up electron / muon trigger SFs
e_trigs = {
    "2016preVFP": "Ele25_EtaLt2p1",
    "2016postVFP": "Ele25_EtaLt2p1",
    "2017": "Ele35",
    "2018": "Ele35",
}
e_trig_base = f"../corrections/electron_trigger/UL_{year}"
e_trig_file = join(e_trig_base, f"Electron_RunUL{year}_{e_trigs[year]}.root")
e_trig_SFs = get_electron_trigger_SFs(e_trig_file)

m_trigs = {
    "2016preVFP": "IsoMu24orIsoTkMu24",
    "2016postVFP": "IsoMu24orIsoTkMu24",
    "2017": "IsoMu27",
    "2018": "IsoMu27",
}
m_trig_base = f"../corrections/muon_trigger/UL_{year}"
m_trig_file = join(m_trig_base, f"Muon_RunUL{year}_{m_trigs[year]}.root")
m_trig_SFs = get_muon_trigger_SFs(m_trig_file)

# load up btagging tables
btag_root = "../corrections/btag/"
btag_tables = get_btag_tables(btag_root, f"{year}", UL=True)
btag_SFs = get_btag_SFs(btag_root, f"{year}", UL=True)

# load up non-signal MC csv / yaml files
fset_string = f"{source}_{year}"
sample_info = get_sample_info(join("../samples", fset_string + ".csv"))
fileset = get_fileset(join("../samples/filesets", fset_string + ".yaml"))
pileup_weights = None
if "MC" in source or "signal" in source:
    pileup_weights = get_pileup_weights("../corrections/pileup/", year=year)

# only run over root files
for sample, files in fileset.items():
    good_files = []
    for f in files:
        if f.split(".")[-1] == "root":
            good_files.append(f)
    fileset[sample] = good_files
logging.info(f"running on\n {fileset.keys()}")

# extract the sum_of_weights from the ntuples
nevts_dict, dyjets_weights = None, None

if "MC" in source:
    nevts_dict = get_nevts_dict(fileset, year)
    print("fileset keys", fileset.keys())
    if f"DYJetsToLLM-50_{year}" in fileset.keys():
        dyjets_weights = dyjets_stitch_weights(sample_info, nevts_dict, year)
        
# load up signal MC csv / yaml files
fileset = {k: v[:1] for k, v in fileset.items() if "BBAToZhToLLTauTauM250_2018" in k}

logging.info(f"Successfully built sum_of_weights dict:\n {nevts_dict}")
logging.info(f"Successfully built dyjets stitch weights:\n {dyjets_weights}")

# start timer, initiate cluster, ship over files
tic = time.time()

# instantiate processor module
proc_instance = AnalysisProcessor(
    source=source,
    year=year,
    sample_info=sample_info,
    fileset=fileset,
    pileup_weights=pileup_weights,
    lumi_masks=lumi_masks,
    nevts_dict=nevts_dict,
    met_phi_SFs=met_phi_SFs,
    eleID_SFs=eIDs,
    eleES_SFs=eES_SFs,
    muID_SFs=mIDs,
    muES_SFs=mES_SFs,
    tauID_SFs=tIDs,
    e_trig_SFs=e_trig_SFs,
    m_trig_SFs=m_trig_SFs,
    fake_rates=fake_rates,
    dyjets_weights=dyjets_weights,
    btag_eff_tables=btag_tables[0],
    btag_SFs=btag_SFs,
    btag_pt_bins=btag_tables[1],
    btag_eta_bins=btag_tables[2],
    run_fastmtt=True,
    systematic=None,
    same_sign=False,
)

In [ ]:
futures_run = processor.Runner(
    executor=processor.FuturesExecutor(compression=None, workers=1),
    schema=NanoAODSchema,
)

out = futures_run(
    fileset,
    "Events",
    processor_instance=proc_instance,
)

In [ ]:
out["dR"]["BBAToZhToLLTauTauM250"][::sum, ::sum, ::sum, ::sum, :]

In [ ]:
out["met_phi"]["BBAToZhToLLTauTauM250"][::sum, ::sum, ::sum, "JES_up", :]

In [ ]:
fake_rate = False
proc_instance = AnalysisProcessor(sample_info=sample_info,
                                  pileup_tables=pileup_tables,
                                  lumi_masks=lumi_masks,
                                  nevts_dict=nevts_dict,
                                  high_stats=True,
                                  eleID_SFs=eIDs, muID_SFs=mIDs, tauID_SFs=tIDs,
                                  fake_rates=fake_rates,
                                  dyjets_weights=dyjets_weights,
                                  e_trig_SFs=e_trig_SFs, m_trig_SFs=m_trig_SFs,
                                  btag_eff_tables=btag_eff_tables, btag_SFs=btag_SFs,
                                  btag_pt_bins=btag_pt_bins, btag_eta_bins=btag_eta_bins,
                                  run_fastmtt=True, fill_hists=True)

if fake_rate: 
    proc_instance = SS4lFakeRateProcessor(sample_info=sample_info,
                                          pileup_tables=pileup_tables,
                                          mode='tt',
                                          nevts_dict=nevts_dict,
                                          lumi_masks=lumi_masks,
                                          high_stats=True,
                                          eleID_SFs=eIDs,
                                          muID_SFs=mIDs,
                                          tauID_SFs=tIDs,
                                          dyjets_weights=dyjets_weights,
                                          e_trig_SFs=e_trig_SFs, m_trig_SFs=m_trig_SFs)
    to
out = processor.run_uproot_job(
        fileset ,
        treename="Events",
        processor_instance=proc_instance,
        executor=processor.futures_executor,
        executor_args={"schema": NanoAODSchema, 'workers': 1},
        chunksize=25000
)

In [ ]:
import hist
import numpy as np
import awkward as ak

base = '/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2018/DY4JetsToLLM-50'
file = join(base, 'all_DY4JetsToLLM-50_file001_part_1of3_Electrons.root')
events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
met = events.MET
tau = events.Tau
ele = events.Electron
mu = events.Muon

dataset_axis = hist.axis.StrCategory(name="dataset", label="", categories=[], growth=True)
pt_axis = hist.axis.Regular(name="pt", label=r"$p_T$ [GeV]", bins=25, start=0, stop=250 )
cat_axis = hist.axis.StrCategory(name="category", label="", categories=[], growth=True)
dummy_axis = hist.axis.StrCategory(name="dummy", label="",  categories=[], growth=True)
output = hist.Hist(dataset_axis, pt_axis, cat_axis, dummy_axis)

N = 1000
tt = ak.cartesian({'t1': ele[:N], 't2': tau[:N]}, axis=1)
ll = ak.combinations(ele[:N], 2, axis=1, fields=['l1', 'l2'])
eeet = ak.cartesian({'tt': tt, 'll': ll}, axis=1)
eeet = eeet[(ak.argmax(eeet.tt.t1.pt, axis=1, keepdims=True))]
eeet = eeet[~ak.is_none(eeet, axis=1)]
tt = ak.cartesian({'t1': mu[:N], 't2': tau[:N]}, axis=1)
ll = ak.combinations(ele[:N], 2, axis=1, fields=['l1', 'l2'])
eemt = ak.cartesian({'tt': tt, 'll': ll}, axis=1)
eemt = eemt[(ak.argmax(eemt.tt.t1.pt, axis=1, keepdims=True))]
eemt = eemt[~ak.is_none(eemt, axis=1)]
eeet['category'] = 'eeet'
eemt['category'] = 'eemt'
eeet['weights'] = 0.8*np.ones(len(eeet))
eemt['weights'] = 0.9*np.ones(len(eemt))
cands = {'eeet': eeet, 'eemt': eemt}
lltt = ak.concatenate(list(cands.values()), axis=1)

t1, t2 = lltt.tt.t1, lltt.tt.t2
d_phi = t1.phi - t2.phi
d_eta = t1.eta - t2.eta
dR = np.sqrt(d_phi**2 + d_eta**2)
print(np.histogram(ak.flatten(dR)))

lltt = lltt[ak.num(lltt)==1]
output.fill(dataset='test', pt=np.array(ak.flatten(lltt.ll.l1.pt)), 
            category=ak.to_numpy(ak.flatten(lltt.category)),
            dummy='nom', weight=ak.to_numpy(ak.flatten(lltt.weights)))


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('../')
import argparse
import logging
import time
from os.path import join
import warnings
warnings.filterwarnings("ignore")

import uproot
from coffea import processor, util
from coffea.lumi_tools import LumiMask
from coffea.nanoevents import NanoAODSchema, NanoEventsFactory

from coffea import analysis_tools
import awkward as ak
import numpy as np
from azh_analysis.selections.preselections import (
    append_tight_masks,
    build_Z_cand,
    check_trigger_path,
    closest_to_Z_mass,
    count_btags,
    dR_ll,
    dR_lltt,
    filter_MET,
    filter_PV,
    get_baseline_bjets,
    get_baseline_electrons,
    get_baseline_jets,
    get_baseline_muons,
    get_baseline_taus,
    get_lepton_count_veto_masks,
    get_tt,
    highest_LT,
    is_prompt,
    tight_electrons,
    tight_muons,
    trigger_filter,
)
from azh_analysis.utils.parameters import get_categories, get_eras, get_lumis


categories = get_categories()
cat_to_num = {v: k for k, v in categories.items()}

base = '/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2018/DYBJetsToLL_M-50_Zpt-100to200/'
files = os.listdir(base)

for f in files:
    if ".root" not in f: continue
    try:
        events = NanoEventsFactory.from_root(
            join(base, f), schemaclass=NanoAODSchema
        ).events()
    except:
        continue
    print(f)
    
    # grab baseline leptons, apply energy scale shifts                                   
    baseline_e = get_baseline_electrons(events.Electron)
    baseline_m = get_baseline_muons(events.Muon)
    baseline_t = get_baseline_taus(events.Tau)
    MET = events.MET
    MET["pt"] = MET.T1_pt
    MET["phi"] = MET.T1_phi

    # seeds the lepton count veto                                                        
    tight_e = baseline_e[tight_electrons(baseline_e)]
    tight_m = baseline_m[tight_muons(baseline_m)]

    # build ll pairs 
    candidates={}
    for ll_pair in ["ee", "mm"]:
        if (ll_pair[:2] == "ee") and ("_Electrons" not in f):
            continue
        if (ll_pair[:2] == "mm") and ("_Muons" not in f):
            continue

        l = tight_e if (ll_pair == "ee") else tight_m
        ll = ak.combinations(l, 2, axis=1, fields=["l1", "l2"])
        ll = dR_ll(ll)
        ll = build_Z_cand(ll)
        ll = closest_to_Z_mass(ll)

        # apply trigger filter mask                                                      
        mask, tpt1, teta1, tpt2, teta2 = trigger_filter(
            ll, events.TrigObj, ll_pair
        )
        mask = mask & check_trigger_path(events.HLT, "2018", ll_pair)
        ll = ak.fill_none(ll.mask[mask], [], axis=0)
        
        # build tt pairs, combine to form lltt candidates                                
        for cat in ["eeem", "mmem"]:
            if cat[:2] != ll_pair:
                continue

            # build 4l final state                                                       
            tt = get_tt(baseline_e, baseline_m, baseline_t, cat)
            lltt = ak.cartesian({"ll": ll, "tt": tt}, axis=1)
            lltt = dR_lltt(lltt, cat)
            charge = lltt.tt.t1.charge * lltt.tt.t2.charge
            lltt = lltt[charge < 0]
            lltt = highest_LT(lltt, cat, apply_LT_cut=False)
            mask = np.ones(len(lltt), dtype=bool)
            lltt = ak.fill_none(lltt.mask[mask], [], axis=0)
            if len(ak.flatten(lltt)) == 0:
                continue
            print(len(ak.flatten(lltt)))

            # determine which legs passed tight selections                               
            lltt = append_tight_masks(lltt, cat, relaxed=False)
            lltt["cat"] = cat_to_num[cat]
            lltt["MET"] = MET

            # mask non-prompt and non-tight MC events                                
            lltt = lltt[
                is_prompt(lltt, cat)
            ]
            print("after prompt", len(ak.flatten(lltt)))
            lltt = lltt [ 
                lltt.t1_tight & lltt.t2_tight
            ]
            print("after tight", len(ak.flatten(lltt)))
            if len(ak.flatten(lltt)) == 0:
                continue
                    
            # append the candidates                                                      
            candidates[cat] = lltt

    if len(candidates)==0:
        continue 
        
    cands = ak.concatenate(list(candidates.values()), axis=1)
    cands, jets, bjets = cands[mask], baseline_j[mask], baseline_b[mask]
    cands = ak.flatten(cands)
    if len(cands) == 0:
        continue

    # get lepton count veto masks                                                        
    lepton_count_veto_masks = get_lepton_count_veto_masks(
        baseline_e[mask], baseline_m[mask], baseline_t[mask]
    )
    veto_mask = np.zeros(len(cands), dtype=bool)
    for cat_str, cat_num in self.cat_to_num.items():
        veto_mask = veto_mask | (
            lepton_count_veto_masks[cat_str] & (cands.cat == cat_num)
        )
    cands = cands[veto_mask]
    jets = jets[veto_mask]
    bjets = bjets[veto_mask]

    # count btags                                                                        
    cands = count_btags(cands, bjets)
    print(len(ak.flatten(cands)))
    

In [ ]:
base = '/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2018/DY4JetsToLLM-50'
file = join(base, 'all_DY4JetsToLLM-50_file001_part_1of3_Electrons.root')
events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
#base = '/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2018/SingleMuon_Run2018A'
#file = join(base, 'all_SingleMuon_Run2018A_file038_part_2of3_Muons.root')
#events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
#tag0 = ak.Array(t1.layout.content.project(1)
    
#l1 = cands.ll.l1
#l1_tags = np.asarray(l1.layout.content.tags)
#for tag in np.unique(l1_tags):
##    tags = l1.layout.content.project(tag)
 #   ID = tags.content.parameter("__record__")

#for cat in ['eemt', 'eeet', 'mmet']:
#    l1 = cands.ll.l1[cands.category==cat]
#    print(ak.type(l1))
#    l1 = l1.layout.content.project(0)
#    print(l1)
    
#cands = ak.concatenate([lltt_eemt, lltt_eeet], axis=1)
#print('cands', ak.num(cands))
#tags = np.asarray(cands.layout.content.tags)
#print(tags)
#for tag in np.unique(tags):
#    mask = (tags==tag)
#    print(np.asarray(cands.layout.content.project(tag)))
#    
#ele = ak.Array(leptons.layout.content.project(0))
#print(ele.dEsigmaUp)

ele = events.Electron
met = events.MET
tau = events.Tau
mu = events.Muon
jet = events.Jet

N=10**6
tt = ak.cartesian({'t1': ele[:N], 't2': tau[:N]}, axis=1)
ll = ak.combinations(ele[:N], 2, axis=1, fields=['l1', 'l2'])
eeet = ak.cartesian({'tt': tt, 'll': ll}, axis=1)
eeet = eeet[(ak.argmax(eeet.tt.t1.pt, axis=1, keepdims=True))]
tt = ak.cartesian({'t1': mu[:N], 't2': tau[:N]}, axis=1)
ll = ak.combinations(ele[:N], 2, axis=1, fields=['l1', 'l2'])
eemt = ak.cartesian({'tt': tt, 'll': ll}, axis=1)
eemt = eemt[(ak.argmax(eemt.tt.t1.pt, axis=1, keepdims=True))]
tt = ak.cartesian({'t1': ele[:N], 't2': tau[:N]}, axis=1)
ll = ak.combinations(mu[:N], 2, axis=1, fields=['l1', 'l2'])
mmet = ak.cartesian({'tt': tt, 'll': ll}, axis=1)
mmet = mmet[(ak.argmax(mmet.tt.t1.pt, axis=1, keepdims=True))]
print('eeet', ak.sum(ak.num(eeet)))
print('eemt', ak.sum(ak.num(eemt)))
print('mmet', ak.sum(ak.num(mmet)))
eemt["category"] = 'eemt'
eeet["category"] = 'eeet'
mmet["category"] = 'mmet'
#cands = ak.concatenate([eemt, eeet, mmet], axis=1)
cands = eemt
cands['met'] = met[:N]
cands = cands[~ak.is_none(cands, axis=1)]
cands = cands[ak.num(cands)>0]    
print(cands)

In [ ]:
from time import time

def apply_eleES(ele, met, syst='nom'):
    if (syst=='nom'):
        return ele, met
    t0 = time()
    pt, eta = ele.pt, ele.eta
    phi, mass = ele.phi, ele.mass
    in_barrel = (abs(eta) < 1.479)
    in_crossover = ((abs(eta) > 1.479) & (abs(eta) < 1.653))
    in_endcap = (abs(eta) > 1.653)
    barrel_shifts = {'up': 1.03, 'down': 0.97}
    crossover_shifts = {'up': 1.04, 'down': 0.96}
    endcap_shifts = {'up': 1.05, 'down': 0.95}
    weights = (in_barrel * barrel_shifts[syst] +
               in_crossover * crossover_shifts[syst] +
               in_endcap * endcap_shifts[syst])
    ele_p4 = ak.zip({'pt': ele.pt, 'eta': ele.eta,
                     'phi': ele.phi, 'mass': ele.mass},
                     with_name='PtEtaPhiMLorentzVector')
    ele_p4_shift = (weights * ele_p4)
    ele_x_diff = (1-weights) * ele.pt * np.cos(ele.phi)
    ele_y_diff = (1-weights) * ele.pt * np.sin(ele.phi)
    met_x = met.pt * np.cos(met.phi) + ele_x_diff
    met_y = met.pt * np.sin(met.phi) + ele_y_diff
    met_p4 = ak.zip({'x': met_x, 'y': met_y,
                     'z': 0, 't': 0}, with_name='LorentzVector')
    met['pt'] = met_p4.pt
    met['phi'] = met_p4.phi
    return ele_p4_shift, met

def apply_eleSmear(ele, met, syst='nom'):
    if (syst=='nom'):
        return ele, met
    shift = ele.dEsigmaUp if (syst=='up') else ele.dEsigmaDown
    weights = shift + 1.0
    ele_p4 = ak.zip({'pt': ele.pt, 'eta': ele.eta,
                     'phi': ele.phi, 'mass': ele.mass},
                    with_name='PtEtaPhiMLorentzVector')
    ele_p4_shift = (weights * ele_p4)
    ele_x_diff = (1-weights) * ele.pt * np.cos(ele.phi)
    ele_y_diff = (1-weights) * ele.pt * np.sin(ele.phi)
    met_x = met.pt * np.cos(met.phi) + ele_x_diff
    met_y = met.pt * np.sin(met.phi) + ele_y_diff
    met_p4 = ak.zip({'x': met_x, 'y': met_y,
                     'z': 0, 't': 0}, with_name='LorentzVector')
    met['pt'] = met_p4.pt
    met['phi'] = met_p4.phi
    return ele_p4_shift, met_p4

print(ele.pt)
ele_new, met_new = apply_eleSmear(cands.ll.l1, cands.met, 'up')
print(ele_new.pt)

In [ ]:
[[55.8], [71.7], [44.4, 41.4], [68.5, ... 62.7, 20.5], [85.3, 28.4], [99.1, 22.1]]
1: 0.03943324089050293
2: 0.10126161575317383
3: 0.12982583045959473
4: 0.17160296440124512
[[42.2], [65.1], [53.5], [174], [43.9], ... 48.9], [105, 105], [147], [82.7], [96.1]]

# 

In [ ]:
def apply_eleES(ele, met, eleES_shift='nom', eleSmear_shift='nom'):
    # decide ES weights by region of the detector
    in_barrel = (abs(ele.eta) < 1.479)
    in_crossover = ((abs(ele.eta) > 1.479) & (abs(ele.eta) < 1.653))
    in_endcap = (abs(ele.eta) > 1.653)
    barrel_shifts = {'up': 1.03, 'nom': 1.0, 'down': 0.97}
    crossover_shifts = {'up': 1.04, 'nom': 1.0, 'down': 0.96}
    endcap_shifts = {'up': 1.05, 'nom': 1.0, 'down': 0.95}
    eleES_weights = (in_barrel * barrel_shifts[eleES_shift] +
                     in_crossover * crossover_shifts[eleES_shift] +
                     in_endcap * endcap_shifts[eleES_shift])

    # get smearing weights
    if eleSmear_shift=='nom':
        shift = 0
    else:
        shift = ele.dEsigmaUp if (eleSmear_shift=='up') else ele.dEsigmaDown
    eleSmear_weights = shift + 1.0

    ele_p4 = ak.zip({'pt': ele.pt, 'eta': ele.eta,
                     'phi': ele.phi, 'mass': ele.mass},
                     with_name='PtEtaPhiMLorentzVector')

    # apply weights
    weights = eleES_weights * eleSmear_weights
    ele_p4_shift = (weights * ele_p4)
    ele_x_diff = (1-weights) * ele.pt * np.cos(ele.phi)
    ele_y_diff = (1-weights) * ele.pt * np.sin(ele.phi)
    diffs = {'x': ele_x_diff, 'y': ele_y_diffs}
    return ele_p4_shift, diffs




met_x = met.pt * np.cos(met.phi) + ele_x_diff
    met_y = met.pt * np.sin(met.phi) + ele_y_diff
    met_p4 = ak.zip({'x': met_x, 'y': met_y,
                     'z': 0, 't': 0}, with_name='LorentzVector')
    met['pt'] = met_p4.pt
    met['phi'] = met_p4.phi

In [ ]:
a.keys()

In [ ]:
from os.path import join
import awkward as ak
base = '/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2018/BBAToZhToLLTauTauM325'
file = join(base, 'all_BBAToZhToLLTauTauM325_file006_part_1of3_Electrons.root')
events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
ak.concatenate(
    [
        ak.unflatten(events.run, 1), 
        ak.unflatten(events.luminosityBlock, 1), 
        ak.unflatten(events.event, 1),
    ], 
axis=-1).to_numpy()  # event, run

In [ ]:
jet["pt"] = 

In [ ]:
import awkward as ak
from azh_analysis.utils.corrections import get_tau_ID_weights, tau_ID_weight

tID_base = f"../corrections/tau_ID/UL_{year}"
tID_file = join(tID_base, "tau.corr.json")
tIDs = get_tau_ID_weights(tID_file)
tau_ID_weight(taus, tIDs, "eett", syst="nom")

In [ ]:
base = "/uscms_data/d3/jdezoort/AZh_columnar/CMSSW_10_2_9/src/azh_coffea/src/corrections/temp"
for f in os.listdir(base):
    vs_e, vs_j = f.split("_")[-2], f.split("_")[-3]
    f = uproot.open(join(base, f))
    for k in f.keys():
        ks = k.split("_"-)
        dm, year = ks[0], ks[1]
        if "2016" in year: 
            year = year + ks[2]
        else:
            
        

In [ ]:
f.keys()

In [ ]:
sys.path.append('../')
from azh_analysis.utils.corrections import get_tau_ID_weights
tID_base = f"../corrections/tau_ID/UL_{year}"
tID_file = join(tID_base, "tau.corr.json")
tIDs = get_tau_ID_weights(tID_file)
print([k for k in tIDs.keys()])
print(tIDs["DeepTau2017v2p1VSe"])

In [ ]:
from azh_analysis.utils.btag import get_btag_SFs, get_btag_tables
btag_root = "../corrections/btag/"
btag_tables = get_btag_tables(btag_root, f"{year}", UL=True)
btag_SFs = get_btag_SFs(btag_root, f"{year}", UL=True)
print([k for k in btag_tables[0].keys() if "ZZ" in k])

In [ ]:
import numpy as np
f = util.load("../data_UL_2018_OS_03-10.coffea")

In [ ]:
import numpy as np
from coffea import util

f = util.load("../data_UL_2018_OS_03-10.coffea")
print(f)
e = f["evtID"].value
l = f["lumi_block"].value
ids = np.concatenate([
    e.reshape(-1,1), 
    l.reshape(-1,1),
], axis=-1)
unique_ids = np.unique(ids, axis=1)
len(unique_ids) / len(ids)

In [ ]:
e.view(-1,1)

In [ ]:
import correctionlib
from coffea.lookup_tools import extractor

#base = '/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2018/SingleMuon_Run2018A/'
#file = join(base, "all_SingleMuon_Run2018A_file001_part_2of3_Muons.root")
base = '/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2018/DY3JetsToLLM-50'
file = join(base, 'all_DY3JetsToLLM-50_file025_part_3of3_Muons.root')
events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
print(len(events))
events = events[events.MET.T1_pt < 6500]
print(len(events))
met = events.MET
print(met.fields)
met_pt, met_phi = met.pt, met.phi
#npvs, run = events.Pileup.nTrueInt, events.run
npvs, run = events.PV.npvs, events.run

In [ ]:
ceval = correctionlib.CorrectionSet.from_file("../corrections/met/2018/met.json.gz")

In [ ]:
import awkward as ak
from azh_analysis.utils.corrections import apply_met_phi_SFs

met_pt = met.pt[met.pt < 6500]

In [ ]:
apply_met_phi_SFs(ceval, met, npvs, run, True)    

In [ ]:
year = "2017"
files = {
    "2018": '/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2018/ZZTo4L/all_ZZTo4L_file003_part_1of3_Muons.root',
    "2017": '/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2017/ZZTo4LTuneCP5/all_ZZTo4LTuneCP5_file003_part_1of3_Muons.root',
    "2016postVFP": "/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2016/ZZTo4LTuneCP5_postVFP/all_ZZTo4LTuneCP5_postFVP_file003_part_1of3_Muons.root",
    "2016preVFP": "/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2016/ZZTo4L_preVFP/all_ZZTo4L_preVFP_file003_part_1of3_Muons.root",
}
file = files[year]
events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()

In [ ]:
import numpy as np
import awkward as ak

np.unique(events.run)

In [ ]:
import awkward as ak
from matplotlib import pyplot as plt
from azh_analysis.utils.corrections import get_met_phi_weights, apply_met_phi_SFs

# grab met phi weights
met_phi_base = f"../corrections/met/{year}"
met_phi_file = join(met_phi_base, "met.json.gz")
met_phi_SFs = get_met_phi_weights(met_phi_file)
print(met_phi_SFs)

met = events.MET
met["pt"], met["phi"] = met.T1_pt, met.T1_phi
npvs, runs = events.PV.npvsGood, events.run

met_phi_before = ak.to_numpy(met.phi)
met_pt_before = ak.to_numpy(met.pt)
met_after = apply_met_phi_SFs(met_phi_SFs, met, npvs, runs, is_data=False)
met_phi_after = ak.to_numpy(met_after.phi)
met_pt_after = ak.to_numpy(met_after.pt)

print(file)
for i in range(20):
    print(f"event {events.event[i]}; phi: {met_phi_before[i]:.6}\t --> {met_phi_after[i]:.6}\t pt: {met_pt_before[i]:.6f}\t --> {met_pt_after[i]:.6}")

In [ ]:
fig = plt.figure(dpi=250, figsize=(4,3))
plt.hist(met_phi_before, histtype="step", label="before")
plt.hist(met_phi_after, histtype="step", label="after")
plt.legend(loc="best")
plt.title(f"ZZTo4L {year} file003_1of3_Muons")

In [ ]:
events.MET.T1_pt_unclustEnDown[0:10].to_numpy()

In [ ]:
events.MET.T1_pt_unclustEnUp[0:10].to_numpy()

In [ ]:
groups = {
    "2017": {
        "TT": [
            "TTToSemiLeptonic",
            "TTToHadronic",
            "TTTo2L2Nu",
            "ttZJets",
            "TTWJetsToLNu",
        ],
        "ZZ": [
            "GluGluToContinToZZTo2e2tau",
            "GluGluToContinToZZTo2mu2tau",
            "GluGluToContinToZZTo4e",
            "GluGluToContinToZZTo4mu",
            "GluGluToContinToZZTo4tau",
            "ZZTo4L",
            "ZZTo2Q2Lmllmin4p0",
        ],
        "WZ": [
            "WZTo2Q2Lmllmin4p0",
            "WZTo3LNu",
        ],
        "VVV": [
            "WWW4F",
            "WWW4F_ext1",
            "WWZ4F",
            #"WZZTuneCP5",
            "WZZ_ext1",
            "ZZZ",
            #"ZZZTuneCP5_ext1",
        ],
        "SM-H(125)": [
            "GluGluHToTauTauM125",
            "VBFHToTauTauM125",
            "WminusHToTauTauM125",
            "WplusHToTauTauM125",
            "ZHToTauTauM125_ext1",
            "ttHToTauTauM125",
            "GluGluHToWWTo2L2NuM-125",
            "VBFHToWWTo2L2NuM-125",
            "GluGluZHHToWW",
            "GluGluHToZZTo4LM125",
            "HWminusJHToWW",
            "HWplusJHToWW",
            "HZJHToWW",
            "HZJHToWW_ext1",
        ],
    },
    "2016postVFP": {
        "TT": [
            "TTToSemiLeptonic",
            "TTToHadronic",
            "TTTo2L2Nu",
            "ttZJets",
            "TTWJetsToLNu"
        ],
        "ZZ": [
            "GluGluToContinToZZTo2e2tau",
            "GluGluToContinToZZTo2mu2tau",
            "GluGluToContinToZZTo4e",
            "GluGluToContinToZZTo4mu",
            "GluGluToContinToZZTo4tau",
            "ZZTo4L",
            "ZZTo2Q2Lmllmin4p0"
        ],
        "WZ": [
            "WZTo2Q2L",
            "WZTo3LNu",
        ],
        "VVV": [
            "WWW4F",
            "WWW4F_ext1",
            "WWZ4F",
            #"WZZTuneCP5",
            "WZZ_ext1",
            "ZZZ",
            #"ZZZTuneCP5_ext1",
        ],
        "SM-H(125)": [
            "GluGluHToTauTauM125",
            "VBFHToTauTauM125",
            "WminusHToTauTauM125",
            "WplusHToTauTauM125",
            "ZHToTauTauM125_ext1",
            "ttHToTauTauM125",
            "GluGluHToWWTo2L2NuM-125",
            "VBFHToWWTo2L2NuM-125",
            "GluGluZHHToWW",
            "GluGluHToZZTo4LM125",
            "HWminusJHToWW",
            "HWplusJHToWW",
            "HZJHToWW",
            "HZJHToWW_ext1"
        ],
    },
    "2016preVFP": {
        "TT": [
            "TTToSemiLeptonic",
            "TTToHadronic",
            "TTTo2L2Nu",
            "ttZJets",
            "TTWJetsToLNu",
        ],
        "ZZ": [
            "GluGluToContinToZZTo2e2tau",
            "GluGluToContinToZZTo2mu2tau",
            "GluGluToContinToZZTo4e",
            "GluGluToContinToZZTo4mu",
            "GluGluToContinToZZTo4tau",
            "ZZTo4L",
            "ZZTo2Q2L",
        ],
        "WZ": [
            "WZTo2Q2Lmllmin4p0",
            "WZTo3LNu",
        ],
        "VVV": [
            "WWW4F",
            "WWW4F_ext1",
            "WWZ4F",
            #"WZZTuneCP5",
            "WZZ_ext1",
            "ZZZ",
            #"ZZZTuneCP5_ext1",
        ],
        "SM-H(125)": [
            "GluGluHToTauTauM125",
            "VBFHToTauTauM125",
            "WminusHToTauTauM125",
            "WplusHToTauTauM125",
            "ZHToTauTau_ext1",
            "ttHToTauTauM125",
            "GluGluHToWWTo2L2NuM125",
            "VBFHToWWTo2L2NuM-125",
            "GluGluZHHToWW",
            "GluGluHToZZTo4LM125",
            "HWminusJHToWW",
            "HWplusJHToWW",
            "HZJHToWW",
        ]
    }
}

In [ ]:
year = "2016preVFP"
ha = util.load(f"../output_met-phi_before-after/MC_UL_{year}_None_after.coffea")["met_phi"]
hb = util.load(f"../output_met-phi_before-after/MC_UL_{year}_None_before.coffea")["met_phi"]

ha_grouped, hb_grouped = {}, {}
group_year = groups[year]
for group, processes in group_year.items():
    processes_combined = sum([hb[p][::sum, ::sum, ::sum, ::sum, :] for p in hb.keys() if p in processes])
    hb_grouped[group] = processes_combined
for group, processes in group_year.items():
    processes_combined = sum([ha[p][::sum, ::sum, ::sum, ::sum, :] for p in ha.keys() if p in processes])
    ha_grouped[group] = processes_combined

hb_grouped

In [ ]:
from matplotlib import pyplot as plt

fig, axs = plt.subplots(nrows=1, ncols=2, dpi=200, figsize=(8,4), sharey=True)
for process, hist in hb_grouped.items():
    try:
        hist.plot1d(ax=axs[0], label=process, stack=True)
    except: 
        print("skipping", process)
for process, hist in ha_grouped.items():
    try:
        hist.plot1d(ax=axs[1], label=process, stack=True)
    except: 
        print("skipping", process)
        
axs[0].set_xlabel(r"$\phi_\mathrm{MET}$")
axs[0].set_title(f"{year}: "+r"Before $\phi_\mathrm{MET}$ Corrections")
axs[1].set_xlabel(r"$\phi_\mathrm{MET}$")
axs[1].set_title(f"{year}: "+r"After $\phi_\mathrm{MET}$ Corrections")
plt.legend(loc="center right")
plt.show()

In [ ]:
import os
os.listdir("../")

In [ ]:
from coffea import util
import numpy as np
data = util.load("../data_UL_2018_OS_07-23.coffea")
data.keys()

In [ ]:
from matplotlib import pyplot as plt
plt.hist(data["data_m4l_fine_cons"].value, bins=np.linspace(0, 2500, 50))

In [ ]:
hist = sum(v for k, v in data["m4l_reg"].items())
hist["data", ::sum, ::sum, ::sum, "cons", :]

In [ ]:
import numpy as np
run, evtID, lumi_block, cat, m4l = np.array(data["run"].value), np.array(data["evtID"].value), np.array(data["lumi_block"].value), np.array(data["data_cat"].value), np.array(data["data_m4l_fine_cons"].value)
for i in range(len(evtID)):
    print(f"Event {i}\t Run: {run[i]}\t evt_id: {evtID[i]}\t lumi_block: {lumi_block[i]}\t channel: {cat[i]}\t m4l: {m4l[i]}\n")